# **Laboratorio 8**

-  [Mónica Salvatierra - 22249](https://github.com/alee2602)
- [Paula Barillas - 22764](https://github.com/paulabaal12)
- [Derek Arreaga - 22537](https://github.com/FabianKel)

**Link del repositorio:**

https://github.com/FabianKel/LAB8-IA

## **Task 1 - Teoría**

Responda las siguientes preguntas de forma clara y concisa

**1. Investigar el algoritmo AC-3 y su relación con el algoritmo de
backtracking search.**


El algoritmo AC-3 es una técnica utilizada en problemas de
restricciones para reducir los dominios de las variables
eliminando valores inconsistentes. Al igual que el algoritmo de
Backtracking Search, tiene como objetivo mejorar la eficiencia
de la búsqueda al reducir el espacio de soluciones antes de
aplicar métodos más costosos.
El algoritmo AC-3 funciona de la siguiente manera:
1. Convierte restricciones en arco: Se representan las
restricciones binarias como pares de arcos (X -> Y, Y ->X)
2. Se crea una agenda de arcos: Se almacena una lista de arcos
a revisar.
3. Revisión de arcos: Se verifica si cada valor en el dominio de
una variable tiene al menos un valor compatible en la otra
variable. Si un valor no tiene soporte, se elimina.
4. Repetición hasta encontrar estabilidad: Si un dominio
cambia, entonces los arcos relacionados se deben de volver
a revisar y el proceso continúa hasta que no haya más
modificaciones.

**Ventajas:**
- Reduce el espacio de búsqueda, facilitando la solución
del problema.
- Detecta inconsistencias temprano, evitando
exploraciones inútiles.

**Desventajas:**
- No garantiza una solución óptima, solo filtra valores
inconsistentes.
- Puede ser costoso en problemas con grandes dominios o
muchas restricciones.


**2. Defina en sus propias palabras el término “Arc Consistency”**

Arc Consistency es un concepto que se utiliza en problemas de satisfacción de restricciones. Se dice que un arco entre dos variables es consistente cuando, para cada valor en el dominio de la primera variable, existe al menos un valor en el dominio de la segunda que cumple con la restricción que hay entre ellas. Si no se cumple esto, se eliminan los valores que no sirven, de tal manera que se van reduciendo los dominios y se facilita la búsqueda de una solución al problema.

### **Referencias**

1. Boris. (2025, 28 febrero). ARC consistency explained. BorisTheBrave.Com. https://www.boristhebrave.com/2021/08/30/arc-consistency-explained/

2. Satisfacción de restricciones: algoritmo AC-3. (2024, 2 marzo).
https://www.toolify.ai/es/ai-news-es/satisfaccin-de-
restricciones-algoritmo-ac3-2042502


## **Task 2 - CSP con Backtracking, Beam y Local Search**

In [17]:
import time
import random
from itertools import permutations

**Definición de variables y dominio**

In [18]:

exams = ["E1", "E2", "E3", "E4", "E5", "E6", "E7"]

# Dominio = Días disponibles
domain = ["Lunes", "Martes", "Miércoles"]

# Asignación de exámenes por estudiante
students = {
    "A": ["E1", "E2", "E3"],
    "B": ["E3", "E4", "E5"],
    "C": ["E3", "E5", "E6"],
    "D": ["E1", "E5", "E7"]
}


**Definir la restricción**

In [19]:
def constraints(assignment):
    # Restricción 1: Ningún estudiante puede tener más de un examen en un mismo día
    for student, exs in students.items():
        days = [assignment.get(e) for e in exs if e in assignment]
        if len(days) != len(set(days)):
            return False
    
    # Restricción 2: Estudiantes que toman el mismo curso no pueden tener exámenes el mismo día
    for exam1 in exams:
        for exam2 in exams:
            if exam1 != exam2:
                shared_students = [s for s in students if exam1 in students[s] and exam2 in students[s]]
                if shared_students and assignment.get(exam1) == assignment.get(exam2):
                    return False
    
    return True


#### **Algoritmo de Backtracking Search**

In [20]:
def backtracking(assignment={}):
    if len(assignment) == len(exams):
        return assignment
    
    exam = [e for e in exams if e not in assignment][0]
    for day in domain:
        new_assignment = assignment.copy()
        new_assignment[exam] = day
        if constraints(new_assignment):
            result = backtracking(new_assignment)
            if result:
                return result
    return None

#### **Algoritmo de Beam Search**

In [21]:
# Beam Search
def beam_search(k=2):
    beam = [{}]
    
    for exam in exams:
        new_beam = []
        for assignment in beam:
            for day in domain:
                new_assignment = assignment.copy()
                new_assignment[exam] = day
                if constraints(new_assignment):
                    new_beam.append(new_assignment)
        
        beam = sorted(new_beam, key=lambda x: len(set(x.values())), reverse=True)[:k]
    
    return beam[0] if beam else None

#### **Algoritmo de Local Search**

In [22]:
# Local Search
def local_search(iterations=1000):
    current_assignment = {exam: random.choice(domain) for exam in exams}
    
    for _ in range(iterations):
        if constraints(current_assignment):
            return current_assignment
        
        exam = random.choice(exams)
        current_assignment[exam] = random.choice(domain)
    
    return current_assignment if constraints(current_assignment) else None

#### **Comparación de resultados**

In [23]:

# Evaluación
def evaluate():
    start = time.time()
    result_bt = backtracking()
    time_bt = time.time() - start
    
    start = time.time()
    result_bs = beam_search()
    time_bs = time.time() - start
    
    start = time.time()
    result_ls = local_search()
    time_ls = time.time() - start
    
    return {
        "Backtracking": (result_bt, time_bt),
        "Beam Search": (result_bs, time_bs),
        "Local Search": (result_ls, time_ls)
    }

In [24]:
results = evaluate()
for method, (solution, duration) in results.items():
    print(f"{method}: {duration:.5f} seconds")
    print(solution)


Backtracking: 0.00007 seconds
None
Beam Search: 0.00006 seconds
None
Local Search: 0.00029 seconds
{'E1': 'Miércoles', 'E2': 'Lunes', 'E3': 'Martes', 'E4': 'Miércoles', 'E5': 'Lunes', 'E6': 'Miércoles', 'E7': 'Martes'}


#### **Conclusiones**